# Setting up EC2 instances

In [1]:
import boto3
import os
import time
from fabric import Connection
from botocore.exceptions import ClientError
ec2 = boto3.client('ec2')

### Functionalities

In [2]:
instance_ami = 'ami-061eb2b23f9f8839c'
num_instance = 1
tier = 't2.micro'
key_pair = 'prodkey21'
security_group_name = 'sum'
security_desc = 'Security group for Mongo'


In [3]:
# AWS EC2 list instance images
def list_images():
    res = ec2.describe_images(Owners=['self'])
    for img in res['Images']:
        print("Name: ",img['Name'])
        print("Image: ", img['ImageId'])
        print("Description: ", img['Description'])
        print("----")

In [4]:
# AWS EC2 save instance image
def save_instance(ins, name, desc='My new instance'):
    res = ec2.create_image(InstanceId=ins, Name=name, Description=desc)
    print("Created image: ",res['ImageId'])
    print("Waiting for it to be available...")

    # wait for it to be available
    available = 0
    while (not available):
        status = ec2.describe_images(ImageIds=[res['ImageId']])
        img = status['Images'][0]
        available = (img['State'] == 'available')
        time.sleep(1)

In [5]:
# AWS EC2 create key-pair
def create_key_pair(name):
    response = ec2.create_key_pair(
        KeyName=name
    )
    key = response['KeyMaterial']
    f= open(name+".pem","w")
    f.write(key)
    f.close
    os.system('chmod 400 %s.pem' %name)
    print('Key Pair %s Created' %name)
    return response

In [6]:
# AWS EC2 delete key-pair
def delete_key_pair(name):
    ec2.delete_key_pair(KeyName=name)
    os.remove(name+'.pem')
    print('Key Pair %s Deleted' %name)

In [7]:
# AWS EC2 create security group
def create_security_group(name, description, ip_permissions):
    response = ec2.describe_vpcs()
    vpc_id = response.get('Vpcs', [{}])[0].get('VpcId', '')

    response = ec2.create_security_group(GroupName=name,
                                         Description=description,
                                         VpcId=vpc_id)
    security_group_id = response['GroupId']
    print('Security Group Created %s in vpc %s.' % (security_group_id, vpc_id))

    data = ec2.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully Set %s' %data)
    return security_group_id

In [8]:
def remove_security_group_permissions(security_group_id, ip_permissions):
    data = ec2.revoke_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully removed %s' %data)
    return security_group_id

In [9]:
def set_security_group_permissions(security_group_id, ip_permissions):
    data = ec2.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully Set %s' %data)
    return security_group_id

In [10]:
# AWS EC2 delete security group
def delete_security_group(security_group_id):
    response = ec2.delete_security_group(GroupId=security_group_id)
    print('Security Group %s Deleted' %security_group_id)

In [11]:
# AWS EC2 create instances
def create_instances(image, maxCount, instanceType, key, groupID):
    instances = ec2.run_instances(
        ImageId=image,
        MinCount=1,
        MaxCount=maxCount,
        InstanceType=instanceType,
        KeyName=key,
        SecurityGroupIds=[groupID]
    )
    instance_list = []
    for i in instances['Instances']:
        instance_list.append(i['InstanceId'])
    
    print('Instances Created %s' %instance_list)
    return instance_list

In [12]:
# AWS EC2 list instances
def list_ec2_instances():
    instances = {}
    res = ec2.describe_instances()
    for r in res['Reservations']:
        for ins in r['Instances']:
            if ins['State']['Name'] == 'running' or ins['State']['Name'] == 'pending':
                instances[ins['InstanceId']] = ins['PublicIpAddress']
    print('List of active Instances %s' %instances)
    return instances

In [13]:
# AWS EC2 terminate instances
def terminate_instances(instances):
    response = ec2.terminate_instances(
        InstanceIds=instances
    )
    print('Instances Deleted %s' %instances)
    return response

### Create Key Pair

In [14]:
# generate the new key pair
# create_key_pair(key_pair)

# Set up an Ubuntu node EC2 instance

### First set up security group

In [15]:
# set up permissions for the new security group for the instance/node
node_ip_permissions = [{'IpProtocol': 'tcp',
                   'FromPort': 22,
                   'ToPort': 22,
                   'IpRanges': [{
                       'CidrIp': '0.0.0.0/0',
                       'Description': 'SSH'}]},
                  {'IpProtocol': '-1',
                   'IpRanges': [{
                       'CidrIp': '0.0.0.0/0',
                       'Description': 'All'}]},
                  {'IpProtocol': 'tcp',
                   'FromPort': 27017,
                   'ToPort': 27017,
                   'IpRanges': [{
                       'CidrIp':'0.0.0.0/0',
                       'Description': 'MongoDB'}]}]


In [16]:
# create the security group using the previously made permissions
node_groupID = create_security_group(security_group_name, security_desc, node_ip_permissions)
node_groupID

Security Group Created sg-046ed5a3717296276 in vpc vpc-1d78777a.
Ingress Successfully Set {'ResponseMetadata': {'RequestId': '05ea60ba-0ae6-428e-bc58-0ff0d1b17a19', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'content-length': '259', 'date': 'Sun, 22 Dec 2019 15:33:46 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


'sg-046ed5a3717296276'

Now to create the EC2 instance, in this case we try with 1 first by setting max count to 1

In [17]:
# create a new single instance
# we are using the ubuntu AMI for this and using t2.medium
instance_node_list = create_instances(instance_ami, num_instance, tier, key_pair, node_groupID)
instance_node = instance_node_list[0]
instance_node

Instances Created ['i-0b654b8c00d558798']


'i-0b654b8c00d558798'

In [18]:
# get the public IP of the instance
time.sleep(5)
instance_dic = list_ec2_instances()
node_ip = instance_dic[instance_node]
node_ip

List of active Instances {'i-064012ea2e13e959a': '18.141.90.224', 'i-0b654b8c00d558798': '54.255.175.202'}


'54.255.175.202'

# Using Fabric Connection function to run commands on the EC2 instance

In [19]:
c = Connection(
    host=node_ip,
    user="ubuntu",
    connect_kwargs={
        "key_filename": key_pair + ".pem",
    },
)


# Setting up Metadata 

In [20]:
# Transfer Metadata from S3 to the instance

In [21]:
# Installing Mongo into EC2 instance
print('EC2 instance rebooting, please wait...')
print('Approx. waiting time: 2-5 mins')
time.sleep(50)
mongo_ip = '22.22.222' # for example
name_of_db = 'nezukodb'
mongo_url = f'mongodb://{mongo_ip}' # insert mongo url here

# write to your .env file
environment_file = open(".env", 'w')

environment_file.write(f'database_name = {name_of_db}\n')
environment_file.write(f'mongo_url = {mongo_url}\n')

environment_file.close()


c.sudo('apt-get install gnupg')
c.run('wget -qO - https://www.mongodb.org/static/pgp/server-4.2.asc | sudo apt-key add -')
c.run('echo "deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-4.2.list')
c.sudo('apt-get update')
c.sudo('apt-get install -y mongodb-org')
c.sudo('service mongod start')

c.run('wget https://dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com/metadata.bson')
c.run(f'mongorestore --db {name_of_db} metadata.bson')
c.sudo("sed -i 's/127.0.0.1/0.0.0.0/g' /etc/mongod.conf")
c.sudo('service mongod restart')

EC2 instance rebooting, please wait...
Approx. waiting time: 2-5 mins
Reading package lists...
Building dependency tree...
Reading state information...
gnupg is already the newest version (2.2.4-1ubuntu1.2).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


OK
deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 multiverse
Hit:1 http://ap-southeast-1.ec2.archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://ap-southeast-1.ec2.archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ap-southeast-1.ec2.archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:4 https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 InRelease
Get:5 https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 Release [3953 B]
Get:6 https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 Release.gpg [801 B]
Get:7 http://ap-southeast-1.ec2.archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [8570 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ap-southeast-1.ec2.archive.ubuntu.com/ubuntu bionic/universe Translation-en [4941 kB]
Get:10 http://ap-southeast-1.ec2.archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [151 kB]
Get:11 http://ap-southeast-1

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 81.9 MB in 3s (27.6 MB/s)
Selecting previously unselected package mongodb-org-shell.
(Reading database ... 100%56531 files and directories currently installed.)
Preparing to unpack .../mongodb-org-shell_4.2.2_amd64.deb ...
Unpacking mongodb-org-shell (4.2.2) ...
Selecting previously unselected package mongodb-org-server.
Preparing to unpack .../mongodb-org-server_4.2.2_amd64.deb ...
Unpacking mongodb-org-server (4.2.2) ...
Selecting previously unselected package mongodb-org-mongos.
Preparing to unpack .../mongodb-org-mongos_4.2.2_amd64.deb ...
Unpacking mongodb-org-mongos (4.2.2) ...
Selecting previously unselected package mongodb-org-tools.
Preparing to unpack .../mongodb-org-tools_4.2.2_amd64.deb ...
Unpacking mongodb-org-tools (4.2.2) ...
Selecting previously unselected package mongodb-org.
Preparing to unpack .../mongodb-org_4.2.2_amd64.deb ...
Unpacking mongodb-org (4.2.2) ...
Setting up mongodb-org-shell (4.2.2) ...
Setting up mongodb-org-mongos (4.2.2) ...
Setting up mon

--2019-12-22 15:35:09--  https://dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com/metadata.bson
Resolving dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com (dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com)... 52.219.128.59
Connecting to dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com (dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com)|52.219.128.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733780316 (700M) [application/x-www-form-urlencoded]
Saving to: ‘metadata.bson’

     0K .......... .......... .......... .......... ..........  0% 24.1M 29s
    50K .......... .......... .......... .......... ..........  0% 15.8M 37s
   100K .......... .......... .......... .......... ..........  0% 19.8M 36s
   150K .......... .......... .......... .......... ..........  0% 22.6M 35s
   200K .......... .......... .......... .......... ..........  0% 25.2M 33s
   250K .......... .......... .......... .......... ..........  0% 25.0M 33s
   300K .........

...

696150K .......... .......... .......... .......... .......... 97% 74.8M 0s
696200K .......... .......... .......... .......... .......... 97% 64.5M 0s
696250K .......... .......... .......... .......... .......... 97% 75.8M 0s
696300K .......... .......... .......... .......... .......... 97% 70.2M 0s
696350K .......... .......... .......... .......... .......... 97% 73.5M 0s
696400K .......... .......... .......... .......... .......... 97% 61.2M 0s
696450K .......... .......... .......... .......... .......... 97% 4.71M 0s
696500K .......... .......... .......... .......... .......... 97% 4.82M 0s
696550K .......... .......... .......... .......... .......... 97% 58.7M 0s
696600K .......... .......... .......... .......... .......... 97% 65.9M 0s
696650K .......... .......... .......... .......... .......... 97% 81.7M 0s
696700K .......... .......... .......... .......... .......... 97%  215M 0s
696750K .......... .......... .......... .......... .......... 97%  236M 0s
696800K ....

702050K .......... .......... .......... .......... .......... 97% 72.6M 0s
702100K .......... .......... .......... .......... .......... 97% 75.2M 0s
702150K .......... .......... .......... .......... .......... 97% 72.6M 0s
702200K .......... .......... .......... .......... .......... 97% 65.3M 0s
702250K .......... .......... .......... .......... .......... 98% 70.8M 0s
702300K .......... .......... .......... .......... .......... 98% 73.5M 0s
702350K .......... .......... .......... .......... .......... 98% 26.1M 0s
702400K .......... .......... .......... .......... .......... 98%  190M 0s
702450K .......... .......... .......... .......... .......... 98% 85.7M 0s
702500K .......... .......... .......... .......... .......... 98% 68.9M 0s
702550K .......... .......... .......... .......... .......... 98% 72.7M 0s
702600K .......... .......... .......... .......... .......... 98% 62.9M 0s
702650K .......... .......... .......... .......... .......... 98% 73.3M 0s
702700K ....

708000K .......... .......... .......... .......... .......... 98% 59.5M 0s
708050K .......... .......... .......... .......... .......... 98% 75.6M 0s
708100K .......... .......... .......... .......... .......... 98% 73.7M 0s
708150K .......... .......... .......... .......... .......... 98% 27.2M 0s
708200K .......... .......... .......... .......... .......... 98% 71.3M 0s
708250K .......... .......... .......... .......... .......... 98%  232M 0s
708300K .......... .......... .......... .......... .......... 98%  232M 0s
708350K .......... .......... .......... .......... .......... 98%  233M 0s
708400K .......... .......... .......... .......... .......... 98%  179M 0s
708450K .......... .......... .......... .......... .......... 98%  220M 0s
708500K .......... .......... .......... .......... .......... 98%  219M 0s
708550K .......... .......... .......... .......... .......... 98%  236M 0s
708600K .......... .......... .......... .......... .......... 98%  200M 0s
708650K ....

713900K .......... .......... .......... .......... .......... 99% 76.0M 0s
713950K .......... .......... .......... .......... .......... 99% 70.3M 0s
714000K .......... .......... .......... .......... .......... 99% 46.6M 0s
714050K .......... .......... .......... .......... .......... 99%  156M 0s
714100K .......... .......... .......... .......... .......... 99%  223M 0s
714150K .......... .......... .......... .......... .......... 99%  226M 0s
714200K .......... .......... .......... .......... .......... 99%  215M 0s
714250K .......... .......... .......... .......... .......... 99%  214M 0s
714300K .......... .......... .......... .......... .......... 99%  237M 0s
714350K .......... .......... .......... .......... .......... 99%  204M 0s
714400K .......... .......... .......... .......... .......... 99%  195M 0s
714450K .......... .......... .......... .......... .......... 99% 5.01M 0s
714500K .......... .......... .......... .......... .......... 99% 52.7M 0s
714550K ....

<Result cmd="sudo -S -p '[sudo] password: ' service mongod restart" exited=0>